In [ ]:
import os
os.chdir(r'C:\Users\WeiChuanzheng\Desktop\T2T_FC6\SD')

In [ ]:
from collections import Counter
from collections import defaultdict
import numpy as np

In [ ]:
sample = 'HJD'
in_file = f"{sample}.sd.elem.txt"
elem = []
with open(in_file, 'r')as f:
    lines = f.readlines()
for line in lines:
    elem.append(line.rstrip().split())

In [ ]:
# with open ('tmp.txt','w') as f:
#     for k,v in dict(Counter(i[4] for i in elem)).items():
#         f.write(f"{k}\t{v}\n")

In [ ]:
# elem_fil = [i for i in elem if int(i[5])>=1000]

In [ ]:
def merge_intervals(intervals):
    merged = []
    
    chrom_dict = defaultdict(list)
    for chrom, start, end in intervals:
        chrom_dict[chrom].append((start, end))
    
    for chrom, intv_list in chrom_dict.items():
        intv_list.sort(key=lambda x: x[0])
        merged_intervals = []
        current_start, current_end = intv_list[0]
        for start, end in intv_list[1:]:
            if start <= current_end:
                current_end = max(current_end, end)
            else:
                merged_intervals.append((current_start, current_end))
                current_start, current_end = start, end
        merged_intervals.append((current_start, current_end))
        
        for s, e in merged_intervals:
            merged.append((chrom, s, e))
    
    merged.sort(key=lambda x: (x[0], x[1]))
    return merged

In [ ]:
sd_pair = defaultdict(list)
for i in elem:
    key = i[4]
    sd_pair[key].append((i[1], int(i[2]), int(i[3]), i[7]))
sd_pair = dict(sd_pair)
print(len(sd_pair))

In [ ]:
for i in sd_pair.values():
    for j in i:
        if j[0] != 'Chr03':continue
        if abs(j[1] - 18299273) <= 40000:
            print(i)

In [ ]:
sd_pair_fil = {k: v for k, v in sd_pair.items() if len(v) >= 2}
# sd_pair_fil_1000 = {k: v for k, v in sd_pair_fil.items() if any(end - start + 1 >= 1000 for _, start, end, _ in v)}
# print(len(sd_pair_fil_1000))
sd_pair_fil_1000 = {k: v for k, v in sd_pair_fil.items() if all(end - start + 1 >= 1000 for _, start, end, _ in v)}
print(len(sd_pair_fil_1000))

In [ ]:
print(len(Counter(i[4] for i in elem)))
print(len(sd_pair_fil))
print(len(sd_pair_fil_1000))

In [ ]:
def get_sd_total_length(dictionary):
    sd_region = []
    for k,v in dictionary.items():
        for i in v:
            sd_region.append((i[0],min(int(i[1]),int(i[2])),max(int(i[1]),int(i[2]))))
    
    sd_region_merged = merge_intervals(sd_region)
    sd_total_length = sum(end - start + 1 for _, start, end in sd_region_merged)
    
    return sd_total_length
    
def get_sd_mean_length(dictionary):
    sd_region = []
    for k,v in dictionary.items():
        for i in v:
            sd_region.append((i[0],min(int(i[1]),int(i[2])),max(int(i[1]),int(i[2]))))

    sd_region_merged = merge_intervals(sd_region)
    lengths = [end - start + 1 for _, start, end in sd_region_merged]
    sd_mean_length = np.mean(lengths)
    
    return sd_mean_length

In [ ]:
get_sd_total_length(sd_pair_fil_1000)

In [ ]:
get_sd_mean_length(sd_pair_fil_1000)

In [ ]:
inter = {k: v for k, v in sd_pair_fil_1000.items() if len({entry[0] for entry in v}) == 1}
intra = {k: v for k, v in sd_pair_fil_1000.items() if len({entry[0] for entry in v}) > 1}

In [ ]:
print(len(inter))
print(get_sd_total_length(inter))
print(get_sd_mean_length(inter))

In [ ]:
print(len(intra))
print(get_sd_total_length(intra))
print(get_sd_mean_length(intra))

In [ ]:
def get_sd_regions(dictionary):
    sd_region = []
    for k,v in dictionary.items():
        for i in v:
            sd_region.append((i[0],min(int(i[1]),int(i[2])),max(int(i[1]),int(i[2]))))
    
    sd_region_merged = merge_intervals(sd_region)
   
    return sd_region_merged

In [ ]:
out_file = f"{sample}.sd.regions.txt"
with open(out_file, 'w')as f:
    for c,s,e in get_sd_regions(sd_pair_fil_1000):
        f.write(f"{c}\t{s}\t{e}\t{e-s}\n")